<div style="direction:rtl">در این سوال از کتابخانه dlib 
برای تشخیص نقاط روی صورت استفاده شده است.
</div>


In [88]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
import imageio

<div style="direction:rtl">این تابع نقطه متناظر در تصویر دوم برای نقطه داده شده از تصویر اول را محاسبه می کند.</div>

In [89]:
def find_corresponding_point(p1):
  if p1 in vettel_coor.tolist():
    return hamilton_coor[vettel_coor.tolist().index(p1)]
  else:
    return p1


<div style="direction:rtl">مثلث های عکس اول به این تابع داده میشوند، سپس با استفاده از تابع
find_corresponding_point نقاط متناظر رئوس مثلث در عکس دوم محاسبه میشوند. به این ترتیب برای هر مثلث عکس اول یک مثلث متناظر در عکس دوم خواهیم داشت.</div>

In [91]:
def compute_second_image_triangles(triangles):
  second_triangles = []
  for triangle in triangles:
      p1 = [triangle[0], triangle[1]]
      q1 = find_corresponding_point(p1)
      p2 = [triangle[2], triangle[3]]
      q2 = find_corresponding_point(p2)
      p3 = [triangle[4], triangle[5]]
      q3 = find_corresponding_point(p3)
      second_triangles.append([q1[0], q1[1], q2[0], q2[1], q3[0], q3[1]])
  return np.int32(second_triangles)

<div style="direction:rtl">عکس ها خوانده میشوند. vettel عکس اول و hamilton عکس دوم است.
همچنین برای سریع تر اجرا شدن کد عکس ها به سایز 256 * 256 برده میشوند. البته در فایل ها 
res01-1.gif وجود دارد که نتیجه اجرای همین کد بدون تغییر سایز است که تقریبا 4 دقیقه به طول می انجامد. 
عکس اول از این <a href="https://www.formula1.com/en/drivers/sebastian-vettel.html">لینک</a> 
و عکس دوم از این <a href="https://www.formula1.com/en/drivers/lewis-hamilton.html">لینک</a> برداشته شده اند.</div>


In [92]:
vettel = cv2.imread('./vettel.jpg')
hamilton = cv2.imread('./hamilton.jpg')
vettel = cv2.resize(vettel, (256, 256))
hamilton = cv2.resize(hamilton, (256, 256))

<div style="direction:rtl">نقاط روی صورت با استفاده از dlib محاسبه میشوند.
هم چنین مدل استفاده شده را میتوانید از این <a href="https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat">لینک</a> دانلود کنید.
بعد از دانلود باید این فایل را در کنار فایل ipynb یا py قرار دهید.</div>

In [ ]:
p = "./shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

lewis = hamilton.copy()
seb = vettel.copy()
gray_lewis = cv2.cvtColor(lewis, cv2.COLOR_BGR2GRAY)
gray_seb = cv2.cvtColor(seb, cv2.COLOR_BGR2GRAY)
        
rects = detector(gray_lewis, 0)
    
for (i, rect) in enumerate(rects):
    shape = predictor(gray_lewis, rect)
    hamilton_coor = face_utils.shape_to_np(shape)
    
rects = detector(gray_seb, 0)
    
for (i, rect) in enumerate(rects):
    shape = predictor(gray_seb, rect)
    vettel_coor = face_utils.shape_to_np(shape)
    

<div style="direction:rtl">در آرایه vettel_coor نقاط تشخیص داده شده روی صورت وجود دارند. در مرحله بعد با استفاده از این نقاط
و تابع ()getTriangleList از کتابخانه opencv عکس اول مثلث بندی می شود. هم چنین لازم به ذکر است که نقاط گوشه عکس و وسط اضلاع عکس هم به این نقاط به طور دستی برای بهتر شدن نتیجه اضافه میشوند.
در آرایه vettel_triangles مثلث بندیهای مورد نظر را داریم.</div>

In [ ]:
rect = (0, 0, vettel.shape[1], vettel.shape[0])
subdiv  = cv2.Subdiv2D(rect)
for (x, y) in vettel_coor:
  subdiv.insert((x, y))
subdiv.insert((0,0))
subdiv.insert((0, vettel.shape[0]- 1))
subdiv.insert((0, (vettel.shape[0]- 1) // 2))
subdiv.insert((vettel.shape[1] - 1, 0))
subdiv.insert(((vettel.shape[1] - 1) // 2, 0))
subdiv.insert((vettel.shape[1] - 1, vettel.shape[0] - 1))
subdiv.insert(((vettel.shape[1] - 1)// 2, (vettel.shape[0] - 1)// 2))

vettel_triangles = subdiv.getTriangleList()
vettel_triangles = np.int32(vettel_triangles)

<div style="direction:rtl"> از تابع compute_second_image_triangles استفاده میشود تا مثلث های متناظر عکس دوم محاسبه شوند.
50 فریم خواهیم داشت که این 50 فریم با ضرایب t مشخص میشوند. با توجه به مثلث های محاسبه شده برای دو عکس و رابطه 
p_frame = p_1 + (p_2 - p_1) * t که در کلاس در مورد آن صحبت شد، نقاط متناظر و در نتیجه مثلث های متناظر در عکس نتیجه محاسبه میشوند. رئوس مثلث ها در آرایه
frame_triangles نگه داشته می شوند.</div>

In [93]:
hamilton_triangles = compute_second_image_triangles(vettel_triangles)
frames = []
frames.append(vettel)

t = np.arange(0.01, 1, 0.02)
frame_triangles = [vettel_triangles + (hamilton_triangles - vettel_triangles) * tt for tt in t]
frame_triangles = np.int32(frame_triangles)

<div style="direction:rtl">در هر مرحله محاسبه یکی از فریم ها، یکی از مثلث ها در نظر گرفته میشود. با توجه به این مثلث،
مثلث های متناظر در عکس دوم و عکس نتیجه محاسبه میشوند و سپس با استفاده از تبدیل affine هر کدام از عکسها به عکس نتیجه میروند. حال کافی است تنها قسمت مثلثی که در این مرحله محاسبه میشود به عکس نتیجه اضافه شود.
برای این منظور با استفاده از تابع drawContours نقاط مثلث عکس نتیجه به آن داده میشود و هر پیکسل این مثلث برابر با مقدار [1,1,1] قرار داده میشود تا بعد از ضرب این آرایه در عکسی که بعد از تبدیل affine به دست آمده بود، تنها ناحیه مثلثی مورد نظر در تصویر بماند.
از طرفی ممکن است به دلیل اشتراک روی ضلعهای مثلث بعضی از پیکسلهای عکس مقداری بیشتر از 255 پیدا کنند. برای اینکه
از این اتفاق جلوگیری شود تنها جاهایی از دو عکس با هم جمع میشوند که در یکی از عکس ها (عکس نتیجه و عکس تبدیل شده) برابر با صفر باشند. این اتفاق با استفاده از
تابع argwhere کتابحانه numpy انجام میشود.</div>


In [94]:
frame_triangles = np.int32(frame_triangles)
frames = []
frames.append(vettel)
for i in range(len(t)):
    # print(i)
    blank = np.zeros((vettel.shape[0], vettel.shape[1], 3), dtype=np.uint8)
    for j in range(len(frame_triangles[i])):
      p = [[vettel_triangles[j][0], vettel_triangles[j][1]],
         [vettel_triangles[j][2], vettel_triangles[j][3]],
         [vettel_triangles[j][4], vettel_triangles[j][5]]]
      q = [[hamilton_triangles[j][0], hamilton_triangles[j][1]],
              [hamilton_triangles[j][2], hamilton_triangles[j][3]],
              [hamilton_triangles[j][4], hamilton_triangles[j][5]]]
      p = np.float32(p)
      q = np.float32(q)
      temp = np.zeros((vettel.shape[1], vettel.shape[0], 3))
      c = [[frame_triangles[i][j][0], frame_triangles[i][j][1]],
                [frame_triangles[i][j][2], frame_triangles[i][j][3]],
                [frame_triangles[i][j][4], frame_triangles[i][j][5]]]
      c = np.float32(c)
      M = cv2.getAffineTransform(p,c)
      vettel_dst = cv2.warpAffine(vettel,M,(blank.shape[1], blank.shape[0]))
      M = cv2.getAffineTransform(q,c)
      hamilton_dst = cv2.warpAffine(hamilton,M,(blank.shape[1], blank.shape[0]))
      dst = vettel_dst * (1 - t[i]) + hamilton_dst * t[i]
      cc = np.int32(c)
      cv2.drawContours(temp, [cc], 0, (1,1,1), -1)
      dst = dst * temp
      a = np.argwhere(np.logical_and(dst != [0, 0, 0], blank!=[0,0,0]))
      dst[a[:,0], a[:,1]] = [0,0,0]
      blank = blank + dst
    frames.append(blank)

<div style="direction:rtl">همه فریم های محاسبه شده در لیست frames نگه داری میشوند. حال با استفاده از تابع mimsave از کتابخانه
imageio این فریم ها تبدیل به عکس gif میشوند.
مقدار duration داده شده به تابع هم مدت زمانی است که هر فریم نمایش داده میشود. تنها نکته استفاده از imageio این است که این کتابخانه برخلاف opencv عکس ها را به صورت RGB میخواند و لازم است قبل استفاده از تابع mimsave همه عکس ها از  BGR به RGB برده شوند.</div>


In [95]:
frames.append(hamilton)
frames = np.uint8(frames)
rgb = []
for frame in frames:
  f = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  rgb.append(f)
# print(len(frames))
imageio.mimsave('res01.gif', rgb, duration=0.04)
# print("--- %s seconds ---" % (time.time() - start_time))


52
